
<img src="../logos/Icos_cp_Logo_RGB.svg" width="400" align="left"/>

<a id='intro'></a>


# Tools for applying ccg to ICOS data
This notebook includes functions in Python for applying the ccg filters to ICOS data. The notebook is divided in the following parts:

- [Import Python  modules](#import_modules)
- [Time functions](#time)
- [Functions for processing ICOS data](#icos)
- [Help functions for ccgcrv filter](#ccgcrv)
- [Control functions](#control)
- [Widget functions](#widget)
- [Map functions](#map)
- [Plotting functions](#plots)


Use the links to quickly navigate to the parts you are interested in. 

<br>
<a id='import_modules'></a>

## 1. Import modules

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#Import Python modules:
from ccg import ccgfilt
from datetime import datetime as dt
import time
import math
import sys
import requests
import pandas as pd
from pandas.tseries.frequencies import to_offset
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import urllib
import ssl
import os
import shutil
from contextlib import closing
from ipywidgets import interact_manual, ColorPicker, Dropdown, Checkbox, DatePicker, BoundedIntText
from IPython.core.display import display, HTML, clear_output
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
from bokeh.io import output_notebook, reset_output

#Set the notebook as the selected output location:
reset_output()
output_notebook()

%matplotlib inline


#Import ICOS tools:
from icoscp.cpb.dobj import Dobj
from icoscp.sparql import sparqls, runsparql

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='time'></a>


## 2. Time Functions
This part include Python functions that handle time variables.

In [ ]:
def decimalyear_to_datetime(dec_year):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Sat May 09 12:00:00 2020
    Last Changed:     Sat May 09 12:00:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a date expressed as a decimal year as input
                      and converts it to the corresponding datetime object.
                      
    Input parameters: Date expressed as decimal year (e.g. 2015.55553)
                      (var_name: 'dec_year', var_type: Float)

    Output:           String     
    
    """
    
    #Import modules:
    from datetime import datetime, timedelta
    
    #Check if input is float:
    if(isinstance(dec_year, float)):     

        #Get year:
        year = int(dec_year)

        #Get fraction of year:
        rem = dec_year - year

        #Create a datetime object of the start of the year:
        base = datetime(year, 1, 1)

        #Convert decimal fraction of year to datetime and add year:
        result = base + timedelta(seconds=(base.replace(year=base.year + 1) - base).total_seconds() * rem)
        
        #Return result:
        return result.replace(second=0, microsecond=0, minute=0, hour=result.hour)+timedelta(hours=result.minute//30)
    
    #If input is not a float:
    else:
        print('Error! Input is not a float')

In [ ]:
#Function that takes a datetime object as input and returns
#two parameters: current year & fraction of current year (as sec):
def toYearFraction(date):
    
    #Function that returns seconds since epoch
    def sinceEpoch(date): 
        return time.mktime(date.timetuple())
    
    #Seconds since epoch:
    s = sinceEpoch
    
    #Extract year from date:
    year = date.year
    
    #Create date for start of current year:
    startOfThisYear = dt(year=year, month=1, day=1)
    
    #Create date for start of next year:
    startOfNextYear = dt(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration
    
    #Return current year and fraction of current year as seconds:
    return date.year + fraction

In [ ]:
#Function that switches the values of a dataframe with values for one year,
#so that values for the 1st 6 months are refered to the last 6 months & vice versa:
def switch_half_year_periods(df, year, start_year):
    
    #Swirch first 6 months of the year to last 6 months:
    lyrdata = df[(df.index>year) & (df.index<(year+0.5))].copy()
    lyrdata.reset_index(inplace = True)
    lyrdata['datetime'] = lyrdata['datetime']-(year-0.5)
    
    #Swirch last 6 months of the year to first 6 months:
    ryrdata = df[(df.index>(year+0.5)) & (df.index<(year+1))].copy()
    ryrdata.reset_index(inplace = True)
    ryrdata['datetime']=ryrdata['datetime']-(year+0.5)

    
    #Check if input year is the start year:
    #if(year==start_year):
        #ryrdata['Detrended'][ryrdata.count()-1]=np.NaN
    
    #Remove entry for last day of the year:
    if(len(ryrdata)>0):
        ryrdata.loc[len(ryrdata)-1, ('Detrended')]=np.NaN
        
        
    #Concatenate dataframes for first and last 6 months:
    yrdata=pd.concat([ryrdata,lyrdata],ignore_index=True)
    
    #Return dataframe
    return yrdata

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='icos'></a>

## 3. Functions for processing ICOS data
This part includes helping functions for processing ICOS data and 2018 Drought Atmospheric Product data.

In [ ]:
#Function that gets data from ICOS CP for given dobjs:
def get_data(station, t_start, t_end):
    
    #Check if ICOS station has hist, L1 & L2 data:
    if(len(station)>0):
        
        #Check if station has L1 & L2 data:
        if((len(station.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2L2DataObject'])>0) &
           (len(station.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2NrtGrowingDataObject'])>0)):

            #Get date & time for 1st & last observation for L2 product:
            timeStart_L2 = dt.strptime(station.timeStart.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2L2DataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')
            timeEnd_L2 = dt.strptime(station.timeEnd.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2L2DataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')

            #Get date & time for 1st & last observation for L1 product:
            timeStart_L1 = dt.strptime(station.timeStart.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2NrtGrowingDataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')
            timeEnd_L1 = dt.strptime(station.timeEnd.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2NrtGrowingDataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')

            #Get ICOS hist, Level 1 & Level 2 data for selected station:
            icos_df = get_icos_hist_L2_L1(station, timeStart_L2, timeEnd_L2, timeStart_L1, timeEnd_L1)


        #Check if station has L2 data only:
        elif(len(station.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2L2DataObject'])>0):

            #Get date & time for 1st & last observation for L2 product:
            timeStart_L2 = dt.strptime(station.timeStart.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2L2DataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')
            timeEnd_L2 = dt.strptime(station.timeEnd.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2L2DataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')

            #Get ICOS hist & Level 2 data for selected station:
            icos_df = get_icos_hist_L2_L1(station, timeStart_L2, timeEnd_L2)


        #Check if station has L1 data only:
        elif(len(station.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2NrtGrowingDataObject'])>0):

            #Get date & time for 1st & last observation for L2 product:
            timeStart_L1 = dt.strptime(station.timeStart.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2NrtGrowingDataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')
            timeEnd_L1 = dt.strptime(station.timeEnd.loc[station.spec=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2NrtGrowingDataObject'].values[0], '%Y-%m-%dT%H:%M:%SZ')

            #Get ICOS hist & Level 1 data for selected station:
            icos_df = get_icos_hist_L2_L1(station, None, None, timeStart_L1, timeEnd_L1)


        #If station has only hist data (non-ICOS station):  
        else:
            #Get hist data:
            icos_df = get_icos_hist_L2_L1(station)

            
        #Filter data by start and end date:
        icos_df = icos_df.loc[t_start:t_end]
        
        #Check if dataframe is empty:
        if(len(icos_df)==0):
            print("\033[0;31;1m No data available for the selected dates.\033[0;31;0m\n\n")
        
    #No data for selected station:
    else:
        print("\033[0;31;1m No data available for the selected station.\033[0;31;0m\n\n")



    #Return dataframe:
    return icos_df

In [ ]:
def get_icos_hist_L2_L1(station_df, time_start_L2=None, time_end_L2=None, time_start_L1=None, time_end_L1=None): 
    
    '''
    Description: Function that takes a dataframe with data object ids as 
                 input and returns a pandas dataframe with observation 
                 info for an ICOS station. The output includes data from
                 ICOS L1, L2 and historic observations (given that they exist)
                 for one station.
                 
    Input:       1. dataframe with station info for ICOS L1, L2 and historic datasets
                    (var_name: "station_df", var_info: Pandas DataFrame)
                 2. time start for ICOS Level 2 dataset
                    (var_name: "time_start_L2", var_type: datetime object)
                 3. time end for ICOS Level 2 dataset
                    (var_name: "time_end_L2", var_type: datetime object)
                 4. time start for ICOS Level 1 dataset
                    (var_name: "time_start_L1", var_type: datetime object)
                 5. time end for ICOS Level 1 dataset
                    (var_name: "time_end_L1", var_type: datetime object)
    
    Output:      1. dataframe with data
                    (var_type: "pandas dataframe")
    '''
    
    #Get list of data objects for ICOS hist, L1 & L2 data products:
    dobj_L1_L2_ls = [url.replace('https://meta.icos-cp.eu/objects/','')
                     for url in station_df.dobj]

    #Create list to store pandas dataframes with ICOS L1 & L2:
    data_df_ls = []

    #If the station has ICOS L1 & L2 data products:
    if (len(dobj_L1_L2_ls)>0):
        
        #Loop through every dobj in dobj list:
        for dobj in dobj_L1_L2_ls:
            
            #Get pandas dataframe with ICOS L1, L2 or historic data:
            obs_data_df = Dobj(dobj).get().set_index('TIMESTAMP')
            
            #Sort index (dates) in ascending order:
            obs_data_df.sort_index(inplace=True)
            
            
            #Check if dobj corresponds to L2 data product:
            if(station_df.spec.loc[station_df.dobj=='https://meta.icos-cp.eu/objects/'+dobj].values[0]=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2L2DataObject'):
                
                #Filter dataframe by time:
                if(time_start_L1):
                    
                    obs_data_df = obs_data_df[:time_start_L1]
                
                #Filter dataframe by quality flag value:
                obs_data_df = obs_data_df.loc[obs_data_df.Flag=='O']
                
                
            #Check if dobj corresponds to L1 data product:    
            elif(station_df.spec.loc[station_df.dobj=='https://meta.icos-cp.eu/objects/'+dobj].values[0]=='http://meta.icos-cp.eu/resources/cpmeta/atcCo2NrtGrowingDataObject'):
                
                #Filter dataframe by quality flag value:
                obs_data_df = obs_data_df.loc[obs_data_df.Flag=='U']
                
                
            #Check if dobj corresponds to ICOS historic data product:      
            else:
                
                #Filter dataframe by time:
                if(time_start_L2):
                    
                    obs_data_df = obs_data_df[:time_start_L2]
                
                #Filter dataframe by quality flag value:
                obs_data_df = obs_data_df.loc[obs_data_df.Flag=='O']
                
                #Filter dataframe by quality flag value:
                obs_data_df = obs_data_df.loc[obs_data_df.co2>0.0]
                    
                    
            #Add data dataframe of the current data object ID to the list:
            data_df_ls.append(obs_data_df)
            
            

        #Concatenate list of dataframes to one dataframe:
        data_LX_df = pd.concat(data_df_ls)
        
        #Sort the dataframe index in ascending order:
        data_LX_df.sort_index(inplace=True)
        
        #Check if dataframe still includes values after all the filtering:
        if(len(data_LX_df)>0):
            
            #Return dataframe:
            return data_LX_df
        
        else:
            print("\033[0;31;1m No observations available for the current selection of quality flag values!\033[0;31;0m\n\n")
            return pd.DataFrame(None)
        
    else:
        print("\033[0;31;1m No data available for the selected station.\033[0;31;0m\n\n")
        
        return pd.DataFrame(None)
    

In [ ]:
def printmd(string):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 10 12:00:00 2019
    Last Changed:     Fri May 10 12:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that allows you to print the string input parameter using
                      markdown formatting code.
                      
    Input parameters: String of characters
                      (var_name: 'string', var_type: String)

    Output:           String     
    
    """
    
    #import module:
    from IPython.display import Markdown, display
    
    #Enable displaying string with markdown code:
    display(Markdown(string))

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='ccgcrv'></a>

## 4. Help functions for ccgcrv filter
This part includes help functions that set the parameters for the ccgcrv filter functions and organize their output into new formats.

In [ ]:
def run_ccgfilt(df, time_col, var_col, t_zero):
    
    #Function that runs ccgfilter and returns an object with the output.
    
    #Set the default parameters for the ccg filter: 
    shortterm=80
    longterm=667
    sampleinterval=1/24
    numpolyterms=4
    numharmonics=3
    timezero=t_zero
    gap=0
    debug=True 
    
    #Run the ccg filter and return output:
    return ccgfilt.ccgFilter(df[time_col], df[var_col], shortterm, longterm, sampleinterval,
                             numpolyterms, numharmonics, timezero, gap, debug)
    
    

In [ ]:
def get_filt_df(filt_obj):
    
    #Function that returns a dataframe with filtered values.
    
    #Get x,y data for plotting:
    x0 = filt_obj.xinterp
    y1 = filt_obj.getFunctionValue(x0)
    y2 = filt_obj.getPolyValue(x0)
    y3 = filt_obj.getSmoothValue(x0)
    y4 = filt_obj.getTrendValue(x0)
    y5 = y3-y4
    
    #Seasonal Cycle:
    harmonics = filt_obj.getHarmonicValue(x0)
    smooth_cycle = harmonics + filt_obj.smooth - filt_obj.trend
    
    #Get trend of residuals:
    resid_trend = filt_obj.trend
    
    #Get growthrate:
    deriv = filt_obj.deriv
    
    #Add filter output to a pandas dataframe & set datetime as index:
    df = pd.DataFrame({'datetime':x0,'GrowthRate':deriv,'Function':y1,'Poly':y2,'SmoothCurve':y3,
                     'Trend':y4,'Cycle':smooth_cycle, 'Residual':resid_trend,'Detrended':y5}).set_index('datetime')
    
    #Return dataframe:
    return df 

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='control'></a>

## 5. Control functions
This part includes functions that check the user's input from the widgets. The functions control if the selected values are compliant to the restrictions posed by the logic of the filtering methods. Additionally, there are functions that control if there are good quality data (Quality Flag) for the selected station and time period.

In [ ]:
def check_ccgcrv_input(Station, start_date, end_date, time_zero):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 08 14:30:00 2020
    Last Changed:     Fri May 08 14:30:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that checks the user input to widgets and returns a boolean for
                      invalid or empty input.
                      
    Input parameters: 1. List of sublists, where every sublist contains the Station ID & Sampling Height
                         (var_name: 'Station', var_type: List)
                      2. Start date of time period
                         (var_name: 'start_date', var_type: DateTime Object)
                      3. End date of time period
                         (var_name: 'end_date', var_type: DateTime Object)
                      4. value of year for which trend is set to zero 
                         (var_name: 'time_zero', var_type: Int)

    Output:           Pandas DataFrame
    
    """
    
    #Check if NO station has been selected:
    if(len(Station)<1):

        #Print message:
        print(("\033[0;31;1m Select station! \033[0;31;0m\n\n"))

        #Return boolean:
        return False
        
    #Check if a station has been selected:
    else:

        #Check if a start-date and/or end-date have been selected:
        if((start_date==None)|(end_date==None)):

            #Print message:
            print(("\033[0;31;1m Select a start date and/or end date! \033[0;31;0m\n\n"))

            #Return boolean:
            return False

        #If a start-date and an end-date have been selected:
        else:

            #Compute the difference between end_date and start_date:
            diff = end_date - start_date

            #Check if end-date refers to an earlier date than start-date:
            if(diff.days<0):

                #Print message:
                print('\033[0;31;1m Error...\n The selected start-date corresponds to a later date than the selected end-date.\n Enter new dates!\n\n')

                #Return boolean:
                return False
            
            #If the selected start-date & end-date are valid:
            else:
                
                #Check if the start and end date differ by a year:
                if(diff.days<731):
                    
                    #Print message:
                    print('\033[0;31;1m Error...\n The selected start-date and end-date must refer to a time period that is at least 2 years long.\n Enter new dates!\n\n')

                    #Return boolean:
                    return False
                
                #If the selected start-date & end-date are 2 or more years apart:
                else:
                    
                    #Check if year to set trend to zero is within the selected time period:
                    if(((time_zero>start_date.year) & (time_zero<end_date.year))==False):
                        
                        #Print message:
                        print('\033[0;31;1m Error...\n Timezero must refer to a value within the selected time period.\n Enter new value for timezero!\n\n')

                        
                        #Return boolean:
                        return False
                
                    else:
                        #Return boolean:
                        return True

  

In [ ]:
def check_data_for_station(station_id, station_hgh):
    
    #Call function to get file info for ICOS hist, L1 & L2:
    station1 = runsparql.RunSparql(sparql_query=sparqls.icos_hist_L1_L2_sparql(station_id, True), output_format='pandas').run()
    station2 = runsparql.RunSparql(sparql_query=sparqls.icos_hist_L1_L2_sparql(station_id, False), output_format='pandas').run()

    #Concatenate dataframes to one:
    station_df = pd.concat([station1, station2])

    #Check if the station has a registered sampling height:
    if(station_hgh):

        #Filter dataframe to only keep records for selected station height:
        station_df = station_df.loc[station_df.samplingHeight==station_hgh]
        
    #Return dataframe:
    return station_df

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='widget'></a>

## 6. Widget functions
This part includes functions that create the widget forms (e.g. dropdown lists, checkboxes, time-pickers, etc.) for the plots and the interactive map. The user can change the content of the plots or the interactive map by selecting differnt values in the widgets. Additionally, this part also includes a function that helps creating the lables for the station dropdown list.

In [ ]:
#Function that creates the widget form for the plots:
def create_widget_ccgcrv():
    
    #Get pandas dataframe with station info for icos historic datasets (2018 Drought Atmospheric Product):
    hist_st_df = runsparql.RunSparql(sparql_query=sparqls.icos_hist_sparql(), output_format='pandas').run()
    
    #Convert column with "sampling heights" to numeric:
    hist_st_df["height"] = pd.to_numeric(hist_st_df["height"])
    
    #Create widgets:
    tracer = Dropdown(options = ['CO2'])
    station =  Dropdown(options = create_station_labels(hist_st_df.sort_values(['Long_name', 'height'], ascending=[True, True]),
                                                        'Long_name', 'Short_name', 'height'))
    
    #Create dictionary with sampling height info for stations with missing sampling height:
    missing_st_samphgh_dict = dict(zip(['PAL', 'PUI', 'CMN', 'PDM'],['12.0', '84.0', '8.0', '28.0']))
    
    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, start_date, end_date, timezero, color, Daytime, Citation, Save_plot, Export_data):
        
        #Check user input:
        if(check_ccgcrv_input(Station, start_date, end_date, timezero)):
            
            #Check if station has data:
            #special cases for 'JFJ' & 'NOR' that have diff. sampl.heights registered for diff. projects.
            if Station[0]=='JFJ':
                
                #Check if station has data in drought2018 project (hist):
                df1 = check_data_for_station(Station[0], Station[1])
                
                #Check if station has data in ICOS project:
                df2 = check_data_for_station(Station[0], '5.0')

                #Concatenate dataframes to one:
                st_df = pd.concat([df1, df2])


            elif Station[0]=='NOR':
                
                #Check if station has data in drought2018 project (hist) or in ICOS project (L2 data):
                df1 = check_data_for_station(Station[0], Station[1])
                
                #Check if station has data in ICOS project (L1 - NRT data):
                df2 = check_data_for_station(Station[0], '58.0')

                #Concatenate dataframes to one:
                st_df = pd.concat([df1, df2])

            else:

                st_df = check_data_for_station(Station[0], Station[1])
        
            
            #Get data:
            icos_df = get_data(st_df, start_date, end_date)


            #Check if dataframe has data:
            if(icos_df.empty==False):

                #Add column with "hour":
                icos_df['hour']=icos_df.index.hour

                #Change column name from "co2" to "Meas":
                icos_df.rename({'co2':'Meas'}, axis='columns', inplace=True)

                #Add column with Julian Year:
                icos_df['JulDate'] = [toYearFraction(i) for i in icos_df.index]
                
                #If daytime is checked:
                if(Daytime):
                    icos_df=icos_df.query('hour>10')
                    icos_df=icos_df.query('hour<18')
                    
                #print(timezero)
                #print('year_max', icos_df.index.max().year)
                
                
                #Check if timezero is within measurements time period:
                if(timezero<=icos_df.index.max().year):
                    
                    #######################################################
                    ###################       PLOTS      ##################
                    #######################################################
                    
                    #Check if station belongs to stations with missing sampling height:
                    if(Station[0] in missing_st_samphgh_dict.keys()):
                        
                        #Replace "" with sampling height from dictionary:
                        Station[1] = missing_st_samphgh_dict[Station[0]]

                    #Plot observations:
                    plot_obs(icos_df, Station[0], Station[1], Tracer, color, Save_plot)

                    #print(icos_df)
                    print('\n\n')

                    ############################# FILTER #######################
                    #Run ccg filter and get filter object output:
                    filt_obj = run_ccgfilt(icos_df, 'JulDate', 'Meas', timezero)

                    #Get dataframe for with filtered values:
                    filt_df = get_filt_df(filt_obj)

                    #Add column with time expressed as datetime objects:
                    filt_df['time']=[decimalyear_to_datetime(filt_df.index[i]) for i in range(len(filt_df))]
                    ############################################################

                    #Plot monthly means with 2*st dev:
                    plot_ccgfilt_monthly_means(filt_obj, Tracer, Station, Save_plot)

                    #Add space:
                    print('\n\n')

                    #Plot measurements with trend & seasonal signal:
                    plot_meas_trend_seasonalsignal(icos_df, filt_df, Station[0], Station[1], color, Save_plot, Export_data)

                    #Add space:
                    print('\n\n')

                    #Plot detrended values:
                    plot_detrended_bokeh(filt_df, Station[0], Station[1])

                    #Add space:
                    print('\n\n')

                    #Call function to plot monthly means & 2*std of measurements:
                    plot_meas_monthly_mean_std(Station[0], Station[1], icos_df, 'Meas', Save_plot, Export_data)

                    #Add space:
                    print('\n\n')
                    
                    #Call function to plot monthly std dev of measurements:
                    plot_meas_monthly_std(Station[0], Station[1], icos_df, 'Meas', Save_plot)
                    
                    #Add space:
                    print('\n\n')


                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Get a list with citation info for every data object ID:
                        cit_ls = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(st_df.dobj.iloc[i]), output_format='pandas').run().cit[0]
                                  for i in range(len(st_df))]

                        #Print citation title:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                        #Loop through all citations:
                        for cit in cit_ls:

                            #Print data object citation:
                            printmd("<sub>"+cit+"</sub>")
                        
                
                #If timezero refers to a later date than the latest date of the dataset:            
                else:
                    print("\033[0;31;1m "+'Timezero is not within the time period of available measurements.\nNote that the period of available measurements might be shorter\nthan the period refered to by the selected start- & end-date.'+"\033[0;31;0m\n\n")
                    

            #If no data are available for the selected dates and station:
            else:
                print("\033[0;31;1m "+'No data available for the selected station...'+"\033[0;31;0m\n\n")


           
    

    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 start_date=DatePicker(description='Starting Date',disabled=False),
                                 end_date=DatePicker(description='Ending Date',disabled=False),
                                 timezero=BoundedIntText(value=2010, min=1980, max=dt.now().year,
                                                         step=1, description='Timezero:',disabled=False),
                                 color=ColorPicker(concise=False,
                                                   description='Pick a color',
                                                   value='#3973ac',
                                                   disabled=False),
                                 Daytime=Checkbox(value=True, description='Daytime', disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False),
                                 Save_plot=Checkbox(value=False, description='Save plots', disabled=False),
                                 Export_data=Checkbox(value=False, description='Export data', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 220px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[2].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[3].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[4].layout.width = '430px'
    interact_c.widget.children[4].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[5].layout.width = '430px'
    interact_c.widget.children[5].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[6].layout.width = '430px'
    interact_c.widget.children[6].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[7].layout.width = '430px'
    interact_c.widget.children[7].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[8].layout.width = '430px'
    interact_c.widget.children[8].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[9].layout.width = '430px'
    interact_c.widget.children[9].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[10].description = 'Update Plots'
    interact_c.widget.children[10].button_style = 'danger'
    interact_c.widget.children[10].style.button_color = '#3973ac'
    interact_c.widget.children[10].layout.margin = '10px 10px 40px 408px' # top/right/bottom/left

In [ ]:
def create_widget_map():
    
    #Get pandas dataframe with station info for icos historic datasets (2018 Drought Atmospheric Product):
    hist_st_df = runsparql.RunSparql(sparql_query=sparqls.icos_hist_sparql(), output_format='pandas').run()
    
    #Convert column with "sampling heights" to numeric:
    hist_st_df["height"] = pd.to_numeric(hist_st_df["height"])
    
    #Create dropdown list for stations:
    station =  Dropdown(options = create_station_labels(hist_st_df.sort_values(['Long_name', 'height'], ascending=[True, True]),
                                                        'Long_name', 'Short_name', 'height'))
    
    #Create dropdown list for basemaps:
    basemap_wdgt = Dropdown(options = ['Imagery', 'OpenStreetMap'], description='Basemap')

    #Function that calls functions to update the map,
    #based on the selected station:
    def update_plot_func(Station, basemap):
        
        #Show map:
        plotmap(hist_st_df, Station[0], basemap, d_icon='cloud', icon_col='orange')
        
           
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func, Station=station, basemap=basemap_wdgt)
                                 

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 220px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.margin = '2px 2px 2px 220px'
    interact_c.widget.children[2].description = 'Update map'
    interact_c.widget.children[2].button_style = 'danger'
    interact_c.widget.children[2].style.button_color = '#3973ac'
    interact_c.widget.children[2].layout.margin = '10px 10px 40px 408px' # top/right/bottom/left
    

In [ ]:
def create_station_labels(lookup_df, col_station_name, col_station_code, col_sampling_height):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon Apr 01 10:26:00 2019
    Last Changed:     Tue May 12 13:10:00 2020
    Version:          1.3.0
    Author(s):        Karolina
    
    Description:      Return a list of tuples. Everey tuple contains the label for a specific station and a
                      list with the station's 3-character code and sampling height. The first item of every tuple
                      is used as a label in the station dropdown list (i.e. Select or Multi-select widget).
    
    Input parameters: 1. lookup table (var_name: "lookup_df", var_type: Pandas Dataframe)
                      2. column name for station long name (var_name: "col_station_name", var_type: String)
                      3. column name for station code (var_name: "col_station_code", var_type: String)
                      4. column name for station sampling height (var_name: "col_sampling_height", var_type: String)
    
    Output:           List of tuples (e.g. [('Gartow (alt. 30.0)', ['GAT', '30.0']),...]
                      tuple: 
                            1. Constructed Station Label (var_type: String)
                            2. List of two items
                                i. Station 3-character Code (var_type: String)
                                ii. Station Sampling Height (var_type: String)

    """
    
    #Filter the dataframe and get a dataframe of unique combinations of station-names and 
    #corresponding sampling heights:
    df = lookup_df.filter([col_station_name, col_sampling_height, col_station_code]).drop_duplicates(subset=[col_station_name,
                                                                                                                 col_sampling_height,
                                                                                                                 col_station_code])

    #Create dictionary with sampling height info for stations with missing sampling height:
    missing_st_samphgh_dict = dict(zip(['PAL', 'PUI', 'CMN', 'PDM'],['12.0', '84.0', '8.0', '28.0']))
    
    #Get a list of tuples for every station that has provided tracer-data:
    #Every tuple is constructed like: ('Gartow (alt. 30.0)', ['GAT', '30.0'])
    station_labels = [(df[col_station_name].iloc[i]+ " (alt. " + str(df[col_sampling_height].iloc[i]) + "m)",
                       [df[col_station_code].iloc[i], str(df[col_sampling_height].iloc[i])])
                      if(np.isnan(df[col_sampling_height].iloc[i])==False)
                      else (df[col_station_name].iloc[i]+ " (alt. "+missing_st_samphgh_dict[df[col_station_code].iloc[i]]+"m)",
                            [df[col_station_code].iloc[i], ""])
                      for i in range(len(df)) ]
    
    #Return list:
    return station_labels
    

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='map'></a>

## 7. Map function
This part includes the function that creates the interactive map. The map is produced by using the [folium](https://python-visualization.github.io/folium/quickstart.html) library for interactive map visualizations in Python. 

In [ ]:
def plotmap(stations_df, selected_station, basemap, d_icon='cloud', icon_col='orange'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue Feb 04 10:40:00 2020
    Last Changed:     Wed May 06 10:40:00 2020
    Version:          1.1.0
    Author(s):        Karolina
    
    Description:      Function that takes a dataframe containing info about ICOS Stations and the 3-character
                      station code of a selected station as input and returns an interactive Folium Map, with
                      the location of the selected station highlighted in red. 
                      Folium (URL): https://python-visualization.github.io/folium/quickstart.html
                      
    Input parameters: 1. Dataframe with Information regarding ICOS Stations
                         (var_name: 'stations_df', var_type: Pandas Dataframe)
                      2. Station 3-character Code
                         (var_name: 'selected_station', var_type: String)

    Output:           Folium Map (Folium Map Object)
    
    """
    
    #Import modules:
    import folium

    #Create folium map-object:
    m = folium.Map(
        location=[60.07921, 10.0000],
        zoom_start=3)
    
    #Check what type of basemap is selected:
    if(basemap=='Imagery'):
        
        #Create folium map-object:
        m = folium.Map(location=[float(stations_df.loc[stations_df.Short_name==selected_station].lat.values[0]),
                                 float(stations_df.loc[stations_df.Short_name==selected_station].lon.values[0])],
                       zoom_start=6,
                       tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                       attr = 'Esri',
                       name = 'Esri Satellite',
                       overlay = False,
                       control = True)
        
    else:
        
        #Create folium map-object:
        m = folium.Map(
            location=[float(stations_df.loc[stations_df.Short_name==selected_station].lat.values[0]),
                      float(stations_df.loc[stations_df.Short_name==selected_station].lon.values[0])],
            zoom_start=6)

    #Add marker-tooltip:
    tooltip = 'Click to view station info'

    def add_marker(map_obj, station_code, marker_color):

        #Add popup text:
        popup=folium.Popup("""<meta content="text/html; charset=UTF-8"><style>td{padding: 3px;}</style><table>"""+
                           '<tr><td>Name:</td><td><b>'+stations_df.Long_name.loc[stations_df.Short_name==station_code].values[0]+'</b></td></tr>'+
                           '<tr><td>Code:</td><td><b>'+station_code+'</b></td></tr>'+
                           '<tr><td>Country:</td><td><b>'+stations_df.Country.loc[stations_df.Short_name==station_code].values[0]+'</b></td></tr>'+
                           '<tr><td>Latitude:</td><td><b>'+str(stations_df.lat.loc[stations_df.Short_name==station_code].values[0])+'</b></td></tr>'+
                           '<tr><td>Longitude:</td><td><b>'+str(stations_df.lon.loc[stations_df.Short_name==station_code].values[0])+'</b></td></tr></table>',
                           max_width=450)

        #Create marker and add it to the map:
        folium.Marker(location=[float(stations_df.lat.loc[stations_df.Short_name==station_code].values[0]),
                                float(stations_df.lon.loc[stations_df.Short_name==station_code].values[0])],
                      popup=popup,
                      icon=folium.Icon(color=marker_color, icon=d_icon),
                      tooltip=tooltip).add_to(map_obj)


    #Get list of stations (not incl. selected station):
    station_ls = [i for i in stations_df.Short_name.values if i!=selected_station]  

    #Create markers for all stations except selected station:
    [add_marker(m, st, icon_col) for st in station_ls]

    #Add marker for selected station:
    add_marker(m, selected_station, 'darkred')

    #Show map:
    display(m)



<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='plots'></a>

## 8. Plotting functions
This part includes functions that produce the plots that are presented in the ccgcrv notebook.


In [ ]:
def plot_obs(df, station_code, station_samp_height, tracer, color, save_plot=False):
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")
    
    #Create plot:
    fig=plt.figure(figsize=(15,10))
    ax = plt.subplot(111) 
    
    #Add data to plot:
    df.plot(y='Meas', fontsize=12, linewidth=0, color=color, ax=ax,
            figsize=(15,10), marker='+', markersize=3)
    
    #Set x- and y-axis labels:
    plt.xlabel('Time', fontsize=14, labelpad=15)
    plt.ylabel(tracer.translate(SUB)+' [ppm]', fontsize=14, labelpad=10)
    
    #Define legend loction:
    plt.legend(loc='upper left', shadow=True)
    
    #Add grid:
    plt.grid(zorder=0,linestyle='-.',color='grey')
    
    #Add title:
    plt.title(tracer.translate(SUB) + ' measurements ('+station_code+' '+station_samp_height+'m)', fontsize=18)
    
    #Add annotations:
    ax.annotate("© ICOS ERIC", xy=(-0.04, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)

    ax.annotate("CC4BY", xy=(0.915, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)
    
    #Save plot:
    if(save_plot):
        
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, station_code+station_samp_height+'_meas.png')
        plt.savefig(file,dpi=100)
    
    #Show plot
    plt.show()

In [ ]:
def plot_ccgfilt_monthly_means(filt_obj, tracer, station, save_plot=False):
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")
    
    #Get list of tuples with monthly means:
    mm = filt_obj.getMonthlyMeans()
    
    #Convert list of tuples to dataframe:
    MonthlyAv=pd.DataFrame(mm,columns=['Year','Month','Meas','StDev','n'])
    
    #Add column with day-info:
    MonthlyAv['Day']=15
    
    #Add column with datetime objects:
    MonthlyAv['DateTime']=pd.to_datetime(MonthlyAv[['Year','Month','Day']])
    
    #Set "Datetime" as index:
    MonthlyAv.set_index('DateTime', inplace=True)
    
    #Create plot:
    fig=plt.figure(figsize=(15,10))
    ax = plt.subplot(111) 
    ax.fill_between(x=MonthlyAv.index,
                    y1=MonthlyAv['Meas']-2*MonthlyAv['StDev'],
                    y2=MonthlyAv['Meas']+2*MonthlyAv['StDev'])
    MonthlyAv.plot(y='Meas',fontsize=12, ax=ax, color='yellow')
    
    #Add labels for x- & y-axis:
    plt.xlabel('Time', fontsize=14, labelpad=15)
    plt.ylabel(tracer.translate(SUB)+' [ppm]', fontsize=14, labelpad=15)
    
    #Set plot title:
    ax.set_title('Filtered monthly '+tracer.translate(SUB)+' mean and 2*stddev ('+station[0]+' '+station[1]+'m)', fontsize=18)
    
    #Set legend location:
    plt.legend(loc='upper left', shadow=True)
    
    #Add grid:
    ax.grid(zorder=0,linestyle='-.',color='grey')
    
    #Add annotations:
    ax.annotate("© ICOS ERIC", xy=(-0.04, -0.15), xycoords=ax.get_window_extent,
                  xytext=(30,0), textcoords="offset points", fontsize=12)

    ax.annotate("CC4BY", xy=(0.915, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)
    
    #Save plot:
    if(save_plot):
        
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, station[0]+station[1]+'_ccgfilt_monthly_means.png')
        plt.savefig(file,dpi=100)
    
    #Show plot:
    plt.show()

In [ ]:
def plot_meas_trend_seasonalsignal(icos_df, df, station_name, station_hgh, color, save_plot=False, export_data=False):
    
    #Create plot figure and add subplots:
    fig, ax1 = plt.subplots(figsize=(15,10))

    #Set plot title and lables for x- & y-axis:
    ax1.set_title('CO${_2}$ ('+station_name+' '+station_hgh+'m); measurements, trend and seasonal signal', fontsize=18, pad=10)
    ax1.set_xlabel('Time', fontsize=14, labelpad=15)
    ax1.set_ylabel('CO${_2}$ [ppm]', fontsize=14, labelpad=15)

    #Add grid:
    ax1.grid(zorder=0,linestyle='-.',color='grey')

    #Set ticks for y-axis:
    #plt.yticks(np.arange(370, 570, step=10))

    #Add plots:
    ax1.plot(icos_df.index, icos_df.Meas, linewidth=0, color=color, marker='+', markersize=4, label='Meas') #add observations
    ax1.plot(df.time, df.Trend, color='black', linewidth=3, label='Trend') #add trend
    ax1.plot(df.time, df.SmoothCurve, linewidth=2, color='red', label='CO${_2}$ [ppm] smoothed curve') #add smoothcurve of observations

    #Instantiate a second axis that shares the same x-axis:
    ax2 = ax1.twinx()

    #Set lables for 2nd y-axis:
    ax2.set_ylabel('CO${_2}$ [ppm] seasonal signal', fontsize=14, rotation=270, labelpad=35)
    
    #Add plot:
    ax2.plot(df.time, df.Detrended, color='#000099', linewidth=2, label='CO${_2}$ [ppm] seasonal signal')

    #Set ticks for 2nd y-axis:
    #plt.yticks(np.arange(-20, 20, step=4))
    l1 = ax1.get_ylim()
    l2 = ax2.get_ylim()
    f = lambda x : l2[0]+(x-l1[0])/(l1[1]-l1[0])*(l2[1]-l2[0])
    ticks = f(ax1.get_yticks())
    ax2.yaxis.set_major_locator(matplotlib.ticker.FixedLocator(ticks))

    #Set ticks for x-axis:
    #plt.xticks(np.arange(int(df.index.min()), int(df.index.max())+2, step=1))

    #Add legend:
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    #plt.legend(h1+h2, l1+l2, loc=2, fontsize=12)
    plt.legend(h1+h2, l1+l2, fontsize=14, loc='upper center', bbox_to_anchor=(0.5, -0.10), shadow=True, ncol=4)
    
    #Add annotations:
    ax1.annotate("© ICOS ERIC", xy=(-0.04, -0.25), xycoords=ax1.get_window_extent,
                 xytext=(30,0), textcoords="offset points", fontsize=12)

    ax1.annotate("CC4BY", xy=(0.915, -0.25), xycoords=ax1.get_window_extent,
                 xytext=(30,0), textcoords="offset points", fontsize=12)
    
    #Save plot:
    if(save_plot):
        
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, station_name+station_hgh+'_seasonal_signal.png')
        plt.savefig(file,dpi=100)

    #Show plot:
    plt.show()
    
    #Export data:
    if(export_data):
                
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, station_name+str(station_hgh)+'_filtered.csv')
               
        #Save output as csv:
        df.to_csv(file,index=True)
        


In [ ]:
#Function for creating plot with detrended values:
def plot_detrended(df, st_code, st_samp_hgh, save_plot=False):
    
    #Get start- and end-year:
    firstyr=math.floor(df.index.min())
    lastyr=math.ceil(df.index.max())
    
    #Create plot figure:
    fig = plt.figure(figsize=(15,10))
    ax = plt.subplot(111)
    
    #Add data to plot:
    [switch_half_year_periods(df, y, firstyr).plot(x='datetime', y='Detrended',
                                                   ax=ax, label=str(y),
                                                   color='firebrick', linewidth=3.0)
     if(y==(lastyr-1))
     else switch_half_year_periods(df, y, firstyr).plot(x='datetime', y='Detrended', ax=ax, label=str(y), linewidth=1)
     for y in range(firstyr,lastyr)]    
    
    #Set plot title and x- and y-axis labels:
    ax.set_title(st_code +' '+st_samp_hgh+'m', fontsize=18)
    ax.set_xlabel('Time', fontsize=14, labelpad=15)
    ax.set_ylabel('Detrended', fontsize=14, labelpad=10)
    
    #Format ticks for x- & y-axis:
    plt.xticks(np.arange(0, 1, step=1/12), ('Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb','Mar', 'Apr', 'May', 'Jun'))
    #plt.yticks(np.arange(-15, 20, step=1))
    
    #Set axis limits:
    ax.set_xlim(0,1)
    #ax.set_ylim(5,9)
    
    #Add grid:
    ax.grid(zorder=0,linestyle='-.',color='grey')
    
    #Add & format legend:
    h1, l1 = ax.get_legend_handles_labels()
    plt.legend(h1, l1, fontsize=14, loc='upper left', bbox_to_anchor=(0, 1.0), shadow=True, ncol=1)
    
    #Add annotations:
    ax.annotate("© ICOS ERIC", xy=(-0.04, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)

    ax.annotate("CC4BY", xy=(0.915, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)
    
    #Save plot:
    if(save_plot):
        
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, st_code+st_samp_hgh+'_trend.png')
        
        #Save plot to output folder:
        plt.savefig(file,dpi=100)
    
    #Show plot:
    plt.show()

In [ ]:
def get_colormap(num_of_items):
      
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 04 17:00:00 2019
    Last Changed:     Fri Apr 04 17:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes an integer representing the total number of items that should receive
                      a sepparate color and  returns a colormap (i.e. a list of strings, where every string
                      represents a different color in hexadecimal code) with the same amount of colors.
                      The function can return colormaps for 1 - 256 items.
                      
    Input parameters: Number of items to be colored in a sepparate color
                      (var_name: 'num_of_items', var_type: Integer)

    Output:           List of strings (colormap)
    
    """
    
    #Check input:
    if(isinstance(num_of_items, int)):
    
        #import module:
        from bokeh.palettes import all_palettes, Colorblind, magma

        #Check the number of items to be colored (1-2 items):
        if((num_of_items>0) and (num_of_items<3)):
            return ['#2b83ba','#fdae61'] #return colormap selection
        
        #Check the number of items to be colored (3-8 items):
        elif((num_of_items>2) and (num_of_items<9)):
            return all_palettes['Colorblind'][num_of_items] #return colormap selection
        
        #Check the number of items to be colored (9-256 items):
        elif((num_of_items>8) and (num_of_items<257)):
            return magma(num_of_items) #return colormap selection         
        
        #If the number of items exceeds the number 256:
        else:
            print('Error! Number of items to be colored is zero or higher than 256.')
    
    #If the input parameter is not an integer:
    else:
        print('Input is not an integer.')
        

In [ ]:
#Function that creates a Bokeh line-glyph for a given dataframe and adds it to the given figure object:
def create_line_glyph_from_df(fig_obj, df, x_col_name, y_col_name, name, color, linewidth, legend):
    
    #Define Datasets:
    x = df[x_col_name]
    y = df[y_col_name]
    z = pd.to_datetime(df['time'], unit='ms')

    #Create a ColumnDataSource object:
    source = ColumnDataSource( data = {'x':x, 'y':y, 'z':z,} )
    
    
    #Add line  glyph:
    g0 = fig_obj.line('x', 'y', source=source, line_width=linewidth, color=color, name=name)
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend.append((name, [g0]))


In [ ]:
#Function that creates an interactive plot of the "detrended" values per year:
def plot_detrended_bokeh(df, station_id, station_hgh):

    #Create figure:
    p = figure(plot_width=900, plot_height=500,
               title=station_id +' '+station_hgh+'m',
               x_axis_label='Time',
               y_axis_label='Detrended')

    #Create a list with of years:
    year_ls = list(range(math.floor(df.index.min()), math.ceil(df.index.max())))

    #Create a list of colors for every year:
    col_ls = get_colormap(math.ceil(df.index.max())- math.floor(df.index.min()))

    #Zip the lists and create a dictionary with the years as keys:
    year_color_dict= dict(zip(year_ls,col_ls))

    #Create an empty list that will store the legend info:
    legend_it = []

    #Add column with time expressed as datetime objects:
    df['time']=[decimalyear_to_datetime(df.index[i]) for i in range(len(df))]

    #Add data to plot:
    [create_line_glyph_from_df(p, switch_half_year_periods(df, y, math.floor(df.index.min())), 'datetime', 'Detrended', str(y), year_color_dict[y], 2.5, legend_it)
     if(y==(math.ceil(df.index.max())-1))
     else create_line_glyph_from_df(p, switch_half_year_periods(df, y, math.floor(df.index.min())), 'datetime', 'Detrended', str(y), year_color_dict[y], 1, legend_it)
     for y in range(math.floor(df.index.min()),math.ceil(df.index.max()))] 


    p.xaxis.ticker = [0, 0.0833, 2*0.0833, 3*0.0833, 4*0.0833, 5*0.0833, 6*0.0833, 7*0.0833, 8*0.0833, 9*0.0833, 10*0.0833, 11*0.0833]
    p.xaxis.major_label_overrides = {0: 'Jul', 0.0833: 'Aug', 2*0.0833: 'Sep', 3*0.0833: 'Oct', 4*0.0833:'Nov', 5*0.0833:'Dec',
                                     6*0.0833: 'Jan', 7*0.0833: 'Feb', 8*0.0833: 'Mar', 9*0.0833: 'Apr', 10*0.0833:'May', 11*0.0833:'Jun'}
    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Time (UTC)','@z{%F}'),
        ('Detrended value','@y{0.f}'),
    ],
        formatters={
                '@z'      : 'datetime', # use 'datetime' formatter for 'date' field
                },
            # display a tooltip whenever the cursor is vertically in line with a glyph
            mode='vline'
            ))    


    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Add legend to figure:
    p.add_layout(legend, 'below')

    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Set the license label position:
    label_opts2 = dict(x=770, y=24,
                      x_units='screen', y_units='screen')

    #Create a label object (copyright) and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'

    #Create a label object (license) and format it:
    caption2 = Label(text="CC4BY", **label_opts2)
    caption2.text_font_size = '8pt'

    #Add labels to plot:
    p.add_layout(caption1, 'below')
    p.add_layout(caption2, 'below')

    #Show plot:
    show(p)

In [ ]:
def plot_meas_monthly_mean_std(station_code, station_samp_hgh, df, col_name, save_plot=False, export_data=False):
    
    #Calculate monthly mean and std of measurements:
    #icos_monm_df = df[col_name].resample('M', loffset=pd.Timedelta(15, 'd')).mean()
    #icos_monstd_df = df[col_name].resample('M', loffset=pd.Timedelta(15, 'd')).std()
    icos_monm_df = df[col_name].resample('M').mean()
    icos_monm_df.index = icos_monm_df.index + to_offset(pd.Timedelta(15, 'd'))
    icos_monstd_df = df[col_name].resample('M').std()
    icos_monstd_df.index = icos_monstd_df.index + to_offset(pd.Timedelta(15, 'd'))
    

    #Create plot figure:
    fig=plt.figure(figsize=(15,10))
    ax = plt.subplot(111) 

    #Add grid:
    ax.grid(zorder=0,linestyle='-.',color='grey')

    #Set plot title:
    ax.set_title(label='Measurements ('+station_code+' '+station_samp_hgh+'m) - monthly CO${_2}$ mean and 2*stddev', fontsize=18)

    #Plot envelope:
    ax.fill_between(x=icos_monm_df.index.values,
                    y1=icos_monm_df.values-2*icos_monstd_df.values,
                    y2=icos_monm_df.values+2*icos_monstd_df.values)

    #Plot monthly mean measurements:
    ax.plot(icos_monm_df.index.values,icos_monm_df.values, color='yellow')

    #Set labels for x- & y-axis:
    plt.xlabel('Time', fontsize=14, labelpad=15)
    plt.ylabel('CO2 [ppm]', fontsize=14, labelpad=15)
    
    #Add annotations:
    ax.annotate("© ICOS ERIC", xy=(-0.04, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)

    ax.annotate("CC4BY", xy=(0.915, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)
    
    #Save plot:
    if(save_plot):
        
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, station_code+station_samp_hgh+'_meas_monthly_mean.png')
        
        #Save plot to output folder:
        plt.savefig(file,dpi=100)
        
    #Export data:
    if(export_data):
        
        #Concatenate dataframes with monthly mean & std:
        df_monthly_mean_std = pd.concat([icos_monm_df, icos_monstd_df], axis=1, keys=['mean', 'std'])
        
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, station_code+str(station_samp_hgh)+'_monthly_mean_std.csv')
        #file_filtered = os.path.join(folder, station_code+str(station_samp_hgh)+'_filtered.csv')
               
        #Save output as csv:
        df_monthly_mean_std.to_csv(file,index=True)
        #df_monthly_mean_std.to_csv(file_filtered,index=True)
        
        

        
    #Show plot:
    plt.show()

In [ ]:
def plot_meas_monthly_std(station_code, station_samp_hgh, df, col_name, save_plot=False):
    
    #Calculate monthly st. dev of measurements:
    #icos_monstd_df = df[col_name].resample('M', loffset=pd.Timedelta(15, 'd')).std()
    icos_monstd_df = df[col_name].resample('M').std()
    icos_monstd_df.index = icos_monstd_df.index + to_offset(pd.Timedelta(15, 'd'))

    #Create plot figure:
    fig=plt.figure(figsize=(15,10))
    ax = plt.subplot(111) 

    #Add grid:
    ax.grid(zorder=0,linestyle='-.',color='grey')

    #Set plot title:
    ax.set_title(label='Measurements ('+station_code+' '+station_samp_hgh+'m) - monthly CO${_2}$ stddev', fontsize=18)

    #Plot monthly mean measurements:
    ax.plot(icos_monstd_df.index.values,icos_monstd_df.values, color='blue')

    #Set labels for x- & y-axis:
    plt.xlabel('Time', fontsize=14, labelpad=15)
    plt.ylabel('CO2 [ppm]', fontsize=14, labelpad=15)
    
    #Add annotations:
    ax.annotate("© ICOS ERIC", xy=(-0.04, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)

    ax.annotate("CC4BY", xy=(0.915, -0.15), xycoords=ax.get_window_extent,
                xytext=(30,0), textcoords="offset points", fontsize=12)
    
    #Save plot:
    if(save_plot):
        
        #Set output folder:
        folder = os.path.join(os.path.expanduser('~'), 'output/ccg/')
        
        #Check if folder already exists:
        if not os.path.exists(folder):
            
            #Create folder:
            os.makedirs(folder)
        
        #Set full path to output file:
        file = os.path.join(folder, station_code+station_samp_hgh+'_meas_monthly_std.png')
        
        #Save plot to output folder:
        plt.savefig(file,dpi=100)

        
    #Show plot:
    plt.show()

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>